<a href="https://colab.research.google.com/github/Eddythemachine/ziontechhub_hackathon/blob/main/Training_and_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FUNCTIONS

In [ ]:
!wget https://raw.githubusercontent.com/Eddythemachine/ziontechhub_hackathon/main/functions.py

--2025-04-26 14:52:43--  https://raw.githubusercontent.com/Eddythemachine/ziontechhub_hackathon/main/functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21423 (21K) [text/plain]
Saving to: ‘functions.py’

functions.py        100%[===================>]  20.92K  --.-KB/s    in 0.002s  

2025-04-26 14:52:43 (9.79 MB/s) - ‘functions.py’ saved [21423/21423]



# IMPORTING LIBS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import re
import warnings


#Functions
from functions import (
    add_space,
    encode_category_data,
    print_unique_items,
    print_min_and_max_value,
    income,
    age,
    experience,
    relationship_status,
    house_ownership,
    analyze_city_clusters,
    car_ownership,
    profession_vs_risk_flag,
    state_vs_risk_flag,
    analyze_job_years_vs_risk,
    analyze_house_years_vs_risk
)


# IMPORTING DATASET

## Train Set

In [ ]:
!wget https://raw.githubusercontent.com/Eddythemachine/ziontechhub_hackathon/main/your_file.csv
file_path = 'your_file.csv'
df = pd.read_csv(file_path)
df.head()

--2025-04-26 14:54:02--  https://raw.githubusercontent.com/Eddythemachine/ziontechhub_hackathon/main/your_file.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16096255 (15M) [text/plain]
Saving to: ‘your_file.csv’

your_file.csv       100%[===================>]  15.35M  --.-KB/s    in 0.1s    

2025-04-26 14:54:02 (130 MB/s) - ‘your_file.csv’ saved [16096255/16096255]



,Id,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,172392,4588175,49,20,single,rented,no,Secretary,Bilaspur,Chhattisgarh,6,11,0
1,246131,6510821,31,15,single,rented,no,Librarian,Amritsar,Punjab,12,10,1
2,51741,9338129,26,7,single,rented,no,Artist,Farrukhabad,Uttar_Pradesh,4,14,0
3,143394,8053436,42,8,single,rented,no,Chef,Mysore[7][8][9],Karnataka,8,11,1
4,146824,3589662,42,18,single,rented,yes,Technical_writer,Bareilly,Uttar_Pradesh,13,13,0


In [ ]:
df.head(5)

,Id,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,172392,4588175,49,20,single,rented,no,Secretary,Bilaspur,Chhattisgarh,6,11,0
1,246131,6510821,31,15,single,rented,no,Librarian,Amritsar,Punjab,12,10,1
2,51741,9338129,26,7,single,rented,no,Artist,Farrukhabad,Uttar_Pradesh,4,14,0
3,143394,8053436,42,8,single,rented,no,Chef,Mysore[7][8][9],Karnataka,8,11,1
4,146824,3589662,42,18,single,rented,yes,Technical_writer,Bareilly,Uttar_Pradesh,13,13,0


In [ ]:
print(df.shape)

(201600, 13)


In [ ]:
# split the data into features and target
X = df.drop(['Risk_Flag'],axis=1)
y = df['Risk_Flag']

# FINDING MID POINTS OF AGES

In [ ]:
def group_ages_with_midpoints(df, age_col='Age', bin_size=10):
    """
    Groups ages into bins and computes the midpoint of each bin.

    Parameters:
    - df: pandas DataFrame
    - age_col: name of the age column (default 'Age')
    - bin_size: size of each age bin (default 10)

    Returns:
    - df: original DataFrame with two new columns: 'Age_Group' and 'Age_Midpoint'
    """
    # Define age bin edges
    min_age = df[age_col].min()
    max_age = df[age_col].max()
    bins = np.arange(min_age, max_age + bin_size, bin_size)

    # Create age labels (e.g., '20-30') and midpoints
    labels = [f"{bins[i]}-{bins[i+1]-1}" for i in range(len(bins)-1)]
    midpoints = [(bins[i] + bins[i+1] - 1) / 2 for i in range(len(bins)-1)]

    # Cut ages into groups
    df['Age_Group'] = pd.cut(df[age_col], bins=bins, labels=labels, include_lowest=True)
    df['Age_Midpoint'] = pd.cut(df[age_col], bins=bins, labels=midpoints, include_lowest=True)
    df['Age_Midpoint'] = df['Age_Midpoint'].astype(float)  # Ensure numeric type

    return df


In [ ]:
df = group_ages_with_midpoints(df, age_col='Age', bin_size=10)
df[['Age', 'Age_Group', 'Age_Midpoint']].head()

,Age,Age_Group,Age_Midpoint
0,49,41-50,45.5
1,31,21-30,25.5
2,26,21-30,25.5
3,42,41-50,45.5
4,42,41-50,45.5


# ADDING FEATURES TO BOOST ALGORITHM

In [ ]:
# Convert Age_Group like '21-30' to midpoint numeric value
def age_group_to_midpoint(age_group):
    if isinstance(age_group, str) and '-' in age_group:
        start, end = map(int, age_group.split('-'))
        return (start + end) / 2
    return np.nan


df['Age_Midpoint'] = df['Age_Group'].apply(age_group_to_midpoint).astype(float)


# Derived Features using Age_Midpoint
df['Experience_Age_Ratio'] = df['Experience'] / df['Age_Midpoint']
df['Job_Stability'] = df['CURRENT_JOB_YRS'] / (df['Experience'] + 1)
df['House_Stability'] = df['CURRENT_HOUSE_YRS'] / df['Age_Midpoint']
df['Experienced'] = (df['Experience'] > 10).astype(int)

df['Income_to_Exp_Ratio'] = df['Income'] / (df['Experience'] + 1)

# Wealth Index - Weighted combination of income and asset ownership
df['Wealth_Index'] = (
    df['Income'] * df['House_Ownership'].map({'Rented': 0.5, 'Owned': 1, 'Norent_Noown': 0}).fillna(0) +
    df['Car_Ownership'].map({'yes': 1, 'no': 0}).fillna(0)
)

# Stability score of current house relative to adult life duration
df['House_Stability_Score'] = df['CURRENT_HOUSE_YRS'] / (df['Age'] - 18 + 1)

# Categorical Group-based Features
profession_stability = df.groupby('Profession')['CURRENT_JOB_YRS'].mean().to_dict()
df['Profession_Stability'] = df['Profession'].map(profession_stability)

city_income_avg = df.groupby('CITY')['Income'].mean().to_dict()
df['City_Cost_of_Living'] = df['CITY'].map(city_income_avg)

df['Marital_Home_Status'] = df['Married/Single'] + "_" + df['House_Ownership']

state_risk = df.groupby('STATE')['Risk_Flag'].mean().to_dict()
df['State_Risk_Profile'] = df['STATE'].map(state_risk)

# Interactions
df['Income_Age_Interaction'] = df['Income'] * df['Age']
df['Experience_Age_Ratio'] = df['Experience'] / (df['Age'] - 18 + 1)

bins = [0, 5, 15, 50]
labels = ['Early', 'Mid', 'Late']

# Binning the 'Experience' column
df['Career_Stage'] = pd.cut(df['Experience'], bins=bins, labels=labels, right=False)

# Add 'Unknown' to the existing categories
df['Career_Stage'] = df['Career_Stage'].cat.add_categories('Unknown')

# Fill NaN values (if any) with 'Unknown'
df['Career_Stage'] = df['Career_Stage'].fillna('Unknown')

df['Years_Since_First_Job'] = df['Age'] - df['Experience']

**These engineered features enhance our risk assessment model by capturing deeper behavioral and socio-economic patterns that standard variables might miss.**

- **Experience_Age_Ratio, Job_Stability, and House_Stability** help assess life and career consistency, which can signal lower or higher risk.
- **Wealth_Index** and **Income-to-Experience Ratio** combine financial status and earning efficiency, which are key indicators of stability.
- **Profession_Stability** and **City_Cost_of_Living** introduce group-level insights into employment and regional economic context.
- **State_Risk_Profile** reflects the average risk behavior at a state level.
- **Categorical interactions** like **Marital_Home_Status** and **Career_Stage** add socio-contextual insights that improve the model's ability to generalize.
- **Years_Since_First_Job** and **Income_Age_Interaction** capture lifecycle and income maturity.

Together, these features help the model make smarter, more reliable predictions about who is more likely to be high risk.

In [ ]:
#Dropping Age_Group as will already have a age mid_point
df.drop(columns=['Age_Group'], inplace=True)

## Splitting Data Into X and y

In [ ]:
X = df.drop(["Risk_Flag"], axis=1)
y = df["Risk_Flag"]

In [ ]:
print(X.shape)
print(y.shape)

(201600, 27)
(201600,)


In [ ]:
print(X.head())
y.head()

       Id   Income  Age  Experience Married/Single House_Ownership  \
0  172392  4588175   49          20         single          rented   
1  246131  6510821   31          15         single          rented   
2   51741  9338129   26           7         single          rented   
3  143394  8053436   42           8         single          rented   
4  146824  3589662   42          18         single          rented   

  Car_Ownership        Profession             CITY          STATE  ...  \
0            no         Secretary         Bilaspur   Chhattisgarh  ...   
1            no         Librarian         Amritsar         Punjab  ...   
2            no            Artist      Farrukhabad  Uttar_Pradesh  ...   
3            no              Chef  Mysore[7][8][9]      Karnataka  ...   
4           yes  Technical_writer         Bareilly  Uttar_Pradesh  ...   

   Income_to_Exp_Ratio  Wealth_Index  House_Stability_Score  \
0         2.184845e+05           0.0               0.343750   
1       

,Risk_Flag
0,0
1,1
2,0
3,1
4,0


### Spliting Data into Train and Test Set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Print the shapes of train and test sets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (161280, 27)
X_test shape: (40320, 27)
y_train shape: (161280,)
y_test shape: (40320,)


### Encoding Categorical Features

In [ ]:
X_train, X_test =  encode_category_data(X_train, X_test)

In [ ]:
X_train.head()

,Id,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,...,Income_to_Exp_Ratio,Wealth_Index,House_Stability_Score,Profession_Stability,City_Cost_of_Living,Marital_Home_Status,State_Risk_Profile,Income_Age_Interaction,Career_Stage,Years_Since_First_Job
57654,157355,6670038,37,13,1,2.0,0,4.0,12.0,0.0,...,4.764313e+05,0.0,0.700000,6.240021,5.480007e+06,5.0,0.117569,246791406,2.0,24
115853,141782,2572120,22,0,1,2.0,0,10.0,4.0,14.0,...,2.572120e+06,0.0,2.200000,6.642607,4.962116e+06,5.0,0.113118,56586640,0.0,22
130470,65328,4341353,78,6,1,0.0,1,36.0,316.0,7.0,...,6.201933e+05,1.0,0.163934,6.441169,5.343853e+06,3.0,0.122163,338625534,2.0,72
39155,246806,3177468,42,17,0,2.0,1,13.0,90.0,0.0,...,1.765260e+05,1.0,0.560000,6.204134,5.144078e+06,2.0,0.117569,133453656,1.0,25
175732,1477,7710970,39,13,1,2.0,0,31.0,33.0,28.0,...,5.507836e+05,0.0,0.636364,5.840186,5.305773e+06,5.0,0.128991,300727830,2.0,26


In [ ]:
from imblearn.over_sampling import SMOTE

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply to training data only
X_resampled, y_train = smote.fit_resample(X_train, y_train)


In [ ]:
# confirm resampling
print(X_resampled.shape)
print(y_train.shape)
print()
print(y_resampled.value_counts())

(282756, 27)
(282756,)

Risk_Flag
0    141378
1    141378
Name: count, dtype: int64


# Using Stander Scaler on Train and Test Set

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit on train set, transform both train and test
x_train_scaled = scaler.fit_transform(X_resampled)
x_test_scaled = scaler.transform(X_test)


In [ ]:
print(x_test_scaled.shape)
x_train_scaled.shape

(40320, 27)


(282756, 27)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

In [ ]:


# Define the algorithms
algorithms = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'AdaBoost': AdaBoostClassifier(),
    'Extra Trees': ExtraTreesClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Train and evaluate each algorithm
results = {}
for name, model in algorithms.items():
    model.fit(x_train_scaled, y_train)
    y_pred = model.predict(x_test_scaled)
    y_pred_proba = model.predict_proba(x_test_scaled)[:, 1] if hasattr(model, "predict_proba") else None

    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba) if y_pred_proba is not None else "N/A"
    report = classification_report(y_test, y_pred)

    results[name] = {
        'Accuracy': accuracy,
        'ROC AUC': roc_auc,
        'Classification Report': report
    }
    print(f"Algorithm: {name}")
    print(f"Accuracy: {accuracy}")
    print(f"ROC AUC: {roc_auc}")
    print("Classification Report:")
    print(report)
    print("-" * 50)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Algorithm: Logistic Regression
Accuracy: 0.6804563492063492
ROC AUC: 0.5680313818551613
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.72      0.80     35466
           1       0.15      0.36      0.21      4854

    accuracy                           0.68     40320
   macro avg       0.52      0.54      0.51     40320
weighted avg       0.80      0.68      0.73     40320

--------------------------------------------------
Algorithm: Random Forest
Accuracy: 0.8942708333333333
ROC AUC: 0.9237640733509146
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.93      0.94     35466
           1       0.55      0.62      0.58      4854

    accuracy                           0.89     40320
   macro avg       0.75      0.77      0.76     40320
weighted avg       0.90      0.89      0.90     40320

--------------------------------------------------
Algorithm: Gradient Boosting

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:31:45] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Algorithm: XGBoost
Accuracy: 0.8811755952380952
ROC AUC: 0.8791778872763834
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.90      0.93     35466
           1       0.50      0.75      0.60      4854

    accuracy                           0.88     40320
   macro avg       0.73      0.83      0.77     40320
weighted avg       0.91      0.88      0.89     40320

--------------------------------------------------


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_dist = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear']
}
model = LogisticRegression()
search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, verbose=1, random_state=42)
search.fit(x_train_scaled, y_train)
best_model = search.best_estimator_

print("Best Params:", search.best_params_ if 'search' in locals() else "No tuning")
print("CV Score:", search.best_score_ if 'search' in locals() else "N/A")

y_pred = best_model.predict(x_test_scaled)
y_pred_proba = best_model.predict_proba(x_test_scaled)[:, 1]

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Params: {'solver': 'lbfgs', 'C': 100}
CV Score: 0.5999308083152964
Test Accuracy: 0.6937996031746032
ROC AUC: 0.5682224862680045
              precision    recall  f1-score   support

           0       0.89      0.74      0.81     35466
           1       0.15      0.34      0.21      4854

    accuracy                           0.69     40320
   macro avg       0.52      0.54      0.51     40320
weighted avg       0.80      0.69      0.74     40320



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
model = RandomForestClassifier()
search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, verbose=1, random_state=42)
search.fit(x_train_scaled, y_train)
best_model = search.best_estimator_

print("Best Params:", search.best_params_ if 'search' in locals() else "No tuning")
print("CV Score:", search.best_score_ if 'search' in locals() else "N/A")

y_pred = best_model.predict(x_test_scaled)
y_pred_proba = best_model.predict_proba(x_test_scaled)[:, 1]

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
param_dist = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}
model = GradientBoostingClassifier()
search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, verbose=1, random_state=42)
search.fit(x_train_scaled, y_train)
best_model = search.best_estimator_

print("Best Params:", search.best_params_ if 'search' in locals() else "No tuning")
print("CV Score:", search.best_score_ if 'search' in locals() else "N/A")

y_pred = best_model.predict(x_test_scaled)
y_pred_proba = best_model.predict_proba(x_test_scaled)[:, 1]

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))
print(classification_report(y_test, y_pred))


In [ ]:
param_dist = {
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
model = DecisionTreeClassifier()
search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, verbose=1, random_state=42)
search.fit(x_train_scaled, y_train)
best_model = search.best_estimator_

print("Best Params:", search.best_params_ if 'search' in locals() else "No tuning")
print("CV Score:", search.best_score_ if 'search' in locals() else "N/A")

y_pred = best_model.predict(x_test_scaled)
y_pred_proba = best_model.predict_proba(x_test_scaled)[:, 1]

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))
print(classification_report(y_test, y_pred))


In [ ]:
param_dist = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance']
}
model = KNeighborsClassifier()
search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=6, cv=5, scoring='accuracy', n_jobs=-1, verbose=1, random_state=42)
search.fit(x_train_scaled, y_train)
best_model = search.best_estimator_

print("Best Params:", search.best_params_ if 'search' in locals() else "No tuning")
print("CV Score:", search.best_score_ if 'search' in locals() else "N/A")

y_pred = best_model.predict(x_test_scaled)
y_pred_proba = best_model.predict_proba(x_test_scaled)[:, 1]

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))
print(classification_report(y_test, y_pred))


In [ ]:
param_dist = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1]
}
model = AdaBoostClassifier()
search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=6, cv=5, scoring='accuracy', n_jobs=-1, verbose=1, random_state=42)
search.fit(x_train_scaled, y_train)
best_model = search.best_estimator_

print("Best Params:", search.best_params_ if 'search' in locals() else "No tuning")
print("CV Score:", search.best_score_ if 'search' in locals() else "N/A")

y_pred = best_model.predict(x_test_scaled)
y_pred_proba = best_model.predict_proba(x_test_scaled)[:, 1]

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))
print(classification_report(y_test, y_pred))


In [ ]:
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}
model = ExtraTreesClassifier()
search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1, verbose=1, random_state=42)
search.fit(x_train_scaled, y_train)
best_model = search.best_estimator_

print("Best Params:", search.best_params_ if 'search' in locals() else "No tuning")
print("CV Score:", search.best_score_ if 'search' in locals() else "N/A")

y_pred = best_model.predict(x_test_scaled)
y_pred_proba = best_model.predict_proba(x_test_scaled)[:, 1]

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))
print(classification_report(y_test, y_pred))


In [ ]:
param_dist = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}
model = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=9, cv=5, scoring='accuracy', n_jobs=-1, verbose=1, random_state=42)
search.fit(x_train_scaled, y_train)
best_model = search.best_estimator_

print("Best Params:", search.best_params_ if 'search' in locals() else "No tuning")
print("CV Score:", search.best_score_ if 'search' in locals() else "N/A")

y_pred = best_model.predict(x_test_scaled)
y_pred_proba = best_model.predict_proba(x_test_scaled)[:, 1]

print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba))
print(classification_report(y_test, y_pred))
